In [1]:
import config
import tweepy
import pandas as pd
import requests
import datetime

In [2]:
today = datetime.datetime.today()
tomorrow = today + datetime.timedelta(days=1)
dayAfter = today + datetime.timedelta(days=2)
dates = [today.strftime("%d-%m-%Y"),tomorrow.strftime("%d-%m-%Y"),dayAfter.strftime("%d-%m-%Y")]
failedTweet = [1389616776728645636]

In [3]:
clean = False
df = pd.read_csv("UserData.csv")

In [4]:
auth = tweepy.OAuthHandler(config.key,config.scret_key)
auth.set_access_token(config.access_token,config.access_token_secret)
api = tweepy.API(auth)

In [ ]:
df = pd.read_csv("UserData.csv")

sinceID = max(df.TweetID.max(),max(failedTweet))
for mention in tweepy.Cursor(api.mentions_timeline,since_id=sinceID).items():
    splitt = mention.text.split(" ")
    username = mention.user.screen_name
    tweetID = mention.id

    if clean:
        try:
            api.destroy_status(tweetID)
            print("Clean: Deleted")
        except:
            print("Clean: Skipped")
            continue
    else:
        try:
            pin = splitt[2]
            age = int(splitt[3].split("+")[0])
            reply = "@" + username + " You have been successfuly subscribed. I will be reminding you when vaccine slot is available at your pincode " + pin
            api.update_status(reply,tweetID)
            df = df.append({ "Username": username,"TweetID": tweetID, "Pin": pin, "Age": age},ignore_index=True)
            print(pin,age,username,tweetID)
        except:
            reply = "@" + username + " " + 'There seems to be an error. Please make sure your tweet is in the following format ' + "errorid:" + datetime.datetime.now().strftime("%Y%m%d%H%M%S")
            file = open("ss.jpeg" , "rb")
            r1 = api.media_upload(filename="ss.jpeg",file=file)
            api.update_status(reply,tweetID,media_ids=[r1.media_id_string])
            print("Wrong tweet")
            failedTweet.append(tweetID)
            continue

df.to_csv("UserData.csv",index=False)

In [ ]:
baseURL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByPin?"
df = pd.read_csv("UserData.csv")
for date in dates:
    for pin in df["Pin"].unique():

        completeURL = baseURL+"date="+date + "&pincode=" + str(pin)
        try:
            sessions = requests.get(completeURL).json()["sessions"]
        except:
            print("Failed to get new sessions")

        if sessions:
            vaccines = {"COVISHIELD" : False, "COVAXIN" : False}
            minAge = {18:False,45:False}
            availableCapacity = sys.maxsize

            for session in sessions:
                availableCapacity = min(availableCapacity,int(session["available_capacity"]))
                vaccines[session["vaccine"]] = True
                minAge[session["min_age_limit"]] = True
            if minAge[18]:
                tweetAt(availableCapacity,vaccines,pin,date,18)
            if minAge[45]:
                tweetAt(availableCapacity,vaccines,pin,date,45)

        else:
            continue

In [118]:
def tweetAt(availableCapacity,vaccines,pin,date,age):

    reply = "Atleast " + str(availableCapacity) + " slot(s) for " + " and ".join([v for v,b in vaccines.items() if b == True]) + " is available for age " + str(age) + "+ at pin " + str(pin) + " for " + date
    newDF = df.dropna()
    tweetIDs = newDF.loc[(df["Age"]==age) & (df["Pin"]==pin),["TweetID"]]

    for tweetID in tweetIDs.TweetID:
        username = df.loc[df["TweetID"]==tweetID,["Username"]].Username.item()
        tweet = "@" + username + " " + reply
        print(tweet)
        api.update_status(tweet,tweetID)

True

In [160]:
alert = df.loc[df["TweetID"]==1389614929078935561,["Alert"]].Alert.item()
pd.isna(alert) or int(datetime.datetime.now().strftime("%d%H%M")) - alert > 100

True

In [146]:
int(datetime.datetime.now().strftime("%d%H%M"))

42306

In [177]:
df.at[df["TweetID"]==1389614929078935562,]

TypeError: unhashable type: 'list'

In [182]:
df.at(df.iloc[df["TweetID"]==1389614929078935562],"Alert")

TypeError: '_AtIndexer' object is not callable

In [226]:

= 4040

In [227]:
df

,Username,TweetID,Pin,Age,Alert
0,get_dash,1389614929078935562,110021,45,4040.0
1,tush3031,1389614929078935561,400706,18,NaN


In [25]:
rel = requests.get("https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByPin?pincode=110012&date=6-05-2021")

In [26]:
rel.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [27]:
rel.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">\n<TITLE>ERROR: The request could not be satisfied</TITLE>\n</HEAD><BODY>\n<H1>403 ERROR</H1>\n<H2>The request could not be satisfied.</H2>\n<HR noshade size="1px">\nRequest blocked.\nWe can\'t connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.\n<BR clear="all">\nIf you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.\n<BR clear="all">\n<HR noshade size="1px">\n<PRE>\nGenerated by cloudfront (CloudFront)\nRequest ID: lSh3GWw9Rryw3-DsVmF9kNBfK3EI6FVs95VmCmdc_QQineDH7vzfpw==\n</PRE>\n<ADDRESS>\n</ADDRESS>\n</BODY></HTML>'

In [28]:
rel.status_code

403

In [5]:
mentions = api.mentions_timeline()

In [8]:
for mention in mentions:
    print("subscribe" in mention.text)

True
True
